In [1]:
import pandas as pd

import numpy as np
import random

import os
from collections import Counter
import pickle

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

import seaborn as sns

from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

from time import time

from gcd_dask_mgmt import extract_delayed_dask_dataframe

from clustering_functions import extract_kmeans_stats

from plot_functions import plot_silhouette_results
from plot_functions import plot_feature_scatter

## Load data

In [2]:
readings_task_usage_df = pd.read_csv("data/sample_jobs_summary_10perc.csv", header=[0,1], index_col=[0])

In [3]:
static_metrics = pd.read_csv('data/static_metrics_and_kmeans.csv')

In [4]:
static_metrics

,job ID,priority,different machines restriction,disk space request - Q1,disk space request - Q2,disk space request - Q3,disk space request - Q4,disk space request - Quartiles,memory request - Q1,memory request - Q2,...,CPU request - Quartiles,priority labels,user,logical job name,scheduling class,K-Means = 2,K-Means = 4,K-Means = 6,K-Means = 8,K-Means = 10
0,3418356,9,0,0,0,0,1,Q4,0,0,...,Q4,Production [9],70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,fGRnr2XEPDr3kQsPccU/k1LELeeQonkj6hDpTP7ALkg=,3,0,1,1,1,1
1,3418405,9,0,0,0,0,1,Q4,0,0,...,Q4,Production [9],70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,q6nwarTUw/Xct0ONQEdblvVhW8uWTquTp8C5la5YfRE=,3,0,1,1,1,1
2,6724949,9,0,0,0,0,1,Q4,0,0,...,Q4,Production [9],70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,4my5Elvc5RumesxoVeuFovkoS28KYA9C3pIBi2bY5Io=,3,0,1,1,1,1
3,28185708,1,0,1,0,0,0,Q1,0,1,...,Q3,"Free [0,1]",WVtO5qw3sNnP4MeiRUnqr07CekrYMU12Mc7GbsgnjhQ=,JGBCBdeRFciFaU6LrbO9Y5w1lBoZ1MyX5Pnx4m05HK8=,2,0,1,1,1,1
4,124371644,9,0,0,0,0,1,Q4,0,0,...,Q3,Production [9],70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,HcYZ4RNZRxmh/W+WuzNBOVk4sOCdDshVEB/McWxfyyk=,3,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64928,6486612269,1,0,0,1,0,0,Q2,0,0,...,Q3,"Free [0,1]",E+9U+J1Dicd5PJklbq2e5THQ29X6D8jmqQ0Zu53Kc+o=,QZm5VyyEiL9TpVhasvtcMxPxENJqlNX4Fn2LdEe+Ff0=,1,1,0,0,0,0
64929,6486612710,4,0,0,0,0,1,Q4,0,0,...,Q2,"Other [2,8]",HxdST/yDI1TlIkr0Povf9KaXGGG8x4iVXX6o/iSIghM=,YI5372ouHbbTv6b594D2bDWC283uv6QcuE41Mst3BFk=,0,1,3,5,5,4
64930,6486630408,4,0,0,1,0,0,Q2,1,0,...,Q3,"Other [2,8]",fJeARInTaIjFrdXGqxap6x2T3TpJB84y9zvFsoGGhjk=,ZoHIMIMjYenKtCPq0iy70XUxyF0JDf1HuW6yUzS4dBM=,0,1,0,0,0,0
64931,6486631154,8,0,0,1,0,0,Q2,0,1,...,Q2,"Other [2,8]",fJeARInTaIjFrdXGqxap6x2T3TpJB84y9zvFsoGGhjk=,cLRsAgA+ajg6giausco6dsV5PxWsqEJZDLdHWhd6v0E=,2,1,0,0,0,0


In [5]:
readings_task_usage_df[readings_task_usage_df['[static_metrics.user == '70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=']]

CPU rate           canonical memory usage            \
                mean       std                   mean       std   
job ID                                                            
3418356     0.036646  0.018963               0.080511  0.000850   
3418405     0.033089  0.023353               0.082067  0.001213   
6724949     0.000554  0.000375               0.078749  0.001790   
28185708    0.000654  0.000272               0.005977  0.001609   
124371644   0.000083  0.000133               0.020451  0.000204   
...              ...       ...                    ...       ...   
6486612269  0.001944  0.002809               0.000313  0.000542   
6486612710  0.021116  0.021414               0.003314  0.003417   
6486630408  0.000629  0.001184               0.000204  0.000413   
6486631154  0.000535  0.000899               0.000249  0.000498   
6486631394  0.002621  0.001527               0.000614  0.000533   

           local disk space usage           active tasks              \
                             mean       std         mean         std   
job ID                                                                 
3418356                  0.000282  0.000072     4.998204    0.052449   
3418405                  0.000218  0.000042     3.996887    0.061824   
6724949                  0.000192  0.000014     1.998922    0.032811   
28185708                 0.000000  0.000000     9.776516    3.269454   
124371644                0.000047  0.000002     1.999162    0.028940   
...                           ...       ...          ...         ...   
6486612269               0.000000  0.000000     1.000000    0.000000   
6486612710               0.000001  0.000001    22.382022  105.542592   
6486630408               0.000000  0.000000     1.285714    0.487950   
6486631154               0.000000  0.000000     1.000000    0.000000   
6486631394               0.000000  0.000000     1.000000    0.000000   

              job runtime  
                     vals  
job ID                     
3418356     2505600000000  
3418405     2505600000000  
6724949     2505600000000  
28185708    2505600000000  
124371644   2505600000000  
...                   ...  
6486612269       24000000  
6486612710      222000000  
6486630408       85000000  
6486631154       63000000  
6486631394       63000000  

[65421 rows x 9 columns]